In [102]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score
from pandas_profiling import ProfileReport
import seaborn as sn
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [103]:
import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine

In [104]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Tshidiso'

In [105]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

In [106]:
sql_query1 = "Select * from claims_data"


In [107]:
df = pd.read_sql_query(sql_query1, connection)

In [109]:
df.head()

,age,sex,bmi,steps,children,smoker,region,insurance_claim,claim_amount
0,19,female,27.9,3009,0,yes,southwest,yes,16884.924
1,18,male,33.77,3008,1,no,southeast,yes,1725.5523
2,28,male,33,3009,3,no,southeast,no,0
3,33,male,22.705,10009,0,no,northwest,no,0
4,32,male,28.88,8010,0,no,northwest,yes,3866.8552


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 9 columns):
age                1338 non-null int64
sex                1338 non-null object
bmi                1338 non-null object
steps              1338 non-null int64
children           1338 non-null object
smoker             1338 non-null object
region             1338 non-null object
insurance_claim    1338 non-null object
claim_amount       1338 non-null object
dtypes: int64(2), object(7)
memory usage: 94.2+ KB


In [111]:
df['bmi'] = pd.to_numeric(df['bmi'])

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 9 columns):
age                1338 non-null int64
sex                1338 non-null object
bmi                1338 non-null float64
steps              1338 non-null int64
children           1338 non-null object
smoker             1338 non-null object
region             1338 non-null object
insurance_claim    1338 non-null object
claim_amount       1338 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 94.2+ KB


In [113]:
y = df.insurance_claim.values
X = df.drop(['insurance_claim','claim_amount'], axis=1)

In [114]:
X_transformed = pd.get_dummies(X, sparse=False, drop_first=True)
X_transformed.head()

,age,bmi,steps,sex_male,children_1,children_2,children_3,children_4,children_5,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,3009,0,0,0,0,0,0,1,0,0,1
1,18,33.770,3008,1,1,0,0,0,0,0,0,1,0
2,28,33.000,3009,1,0,0,1,0,0,0,0,1,0
3,33,22.705,10009,1,0,0,0,0,0,0,1,0,0
4,32,28.880,8010,1,0,0,0,0,0,0,1,0,0


In [115]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.20, random_state = 47)

In [117]:
from sklearn.ensemble import RandomForestClassifier
cls = RandomForestClassifier()

In [118]:
cls.fit(X_train, y_train)

C:\Users\Tshidiso\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [119]:
pred1 = cls.predict(X_test)

In [120]:
print(classification_report(y_test, pred1))

              precision    recall  f1-score   support

          no       0.87      0.94      0.91       102
         yes       0.96      0.92      0.94       166

    accuracy                           0.93       268
   macro avg       0.92      0.93      0.92       268
weighted avg       0.93      0.93      0.93       268



In [121]:
sql_query2 = "Select * from test_set"
df_test = pd.read_sql_query(sql_query2, connection)
df_test.head()

,row_id,age,sex,bmi,steps,children,smoker,region
0,1,27,male,28.88,8049,0,no,southwest
1,2,20,female,26.265,7970,3,yes,southwest
2,3,32,female,26.025,2973,0,no,southwest
3,4,19,female,22.085,10427,1,yes,southwest
4,5,20,female,28.685,3972,0,yes,southwest


In [122]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
row_id      300 non-null object
age         300 non-null int64
sex         300 non-null object
bmi         300 non-null object
steps       300 non-null int64
children    300 non-null object
smoker      300 non-null object
region      300 non-null object
dtypes: int64(2), object(6)
memory usage: 18.9+ KB


In [123]:
df_test['bmi'] = pd.to_numeric(df_test['bmi'])

In [124]:
X_submission = df_test.drop('row_id', axis=1)

In [125]:
X_submission_transformed = pd.get_dummies(X_submission, sparse=False, drop_first=True)
X_submission_transformed.head()

,age,bmi,steps,sex_male,children_1,children_2,children_3,children_4,children_5,smoker_yes,region_northwest,region_southeast,region_southwest
0,27,28.880,8049,1,0,0,0,0,0,0,0,0,1
1,20,26.265,7970,0,0,0,1,0,0,1,0,0,1
2,32,26.025,2973,0,0,0,0,0,0,0,0,0,1
3,19,22.085,10427,0,1,0,0,0,0,1,0,0,1
4,20,28.685,3972,0,0,0,0,0,0,1,0,0,1


In [126]:
predictions = cls.predict(X_submission_transformed)

In [127]:
submission = pd.DataFrame({'row_id' : df_test.row_id,'predictions' : predictions.ravel()}) 

In [128]:
submission.head()

,row_id,predictions
0,1,yes
1,2,no
2,3,yes
3,4,yes
4,5,yes


In [129]:
submission.to_sql('classification_model_results', con=connection, if_exists='append', index=False,method='multi',chunksize=500)

In [130]:
len(submission)

300